# Calendar methods

This tutorial offers a walk through of all the `ExchangeCalendar` methods (for properties see the [calendar properties](./calendar_properties.ipynb) tutorial).

The following sections cover methods according to the nature of the argument(s) they take:
* [Methods that query a Date](#Methods-that-query-a-Date)  
* [Methods that query a Session](#Methods-that-query-a-Session)  
* [Methods that query a Minute](#Methods-that-query-a-Minute)  
* [Methods that query multiple TradingMinute](#Methods-that-query-multiple-TradingMinute)  
* [Methods that query a range of dates](#Methods-that-query-a-range-of-dates)
* [Methods that query a time](#Methods-that-query-a-time)

The following sections cover methods that evaluate an index of trading minutes or sessions:
* [Methods that evaluate an index of contiguous trading minutes](#Methods-that-evaluate-an-index-of-contiguous-trading-minutes)
* [Methods that evaluate an index of contiguous sessions](#Methods-that-evaluate-an-index-of-contiguous-sessions)

`Date` and `Session` refer to types for 'session' parameters (see [sessions.ipynb](./sessions.ipynb) for a tutorial on how to work with sessions).

`Minute` and `TradingMinute` refer to types for 'minute' parameters (see [minutes.ipynb](./minutes.ipynb) for a tutorial on how to work with mintues).

In [1]:
# setup
import exchange_calendars as xcals
import pandas as pd

one_minute = pd.Timedelta(1, "T")

nys = xcals.get_calendar("XNYS") # New York Stock Exchange
hkg = xcals.get_calendar("XHKG") # Hong Kong Stock Exchange

### Methods that query a Date

The argument of methods in this section takes a `Date` type.

In [2]:
# for reference, all times are UTC
nys.schedule.loc["2021-12-31":"2022-01-04"]

,open,break_start,break_end,close
2021-12-31,2021-12-31 14:30:00+00:00,NaT,NaT,2021-12-31 21:00:00+00:00
2022-01-03,2022-01-03 14:30:00+00:00,NaT,NaT,2022-01-03 21:00:00+00:00
2022-01-04,2022-01-04 14:30:00+00:00,NaT,NaT,2022-01-04 21:00:00+00:00


**`is_session`** queries if a date represents a session.

In [3]:
nys.is_session("2022-01-01"), nys.is_session("2022-01-04")

(False, True)

**`date_to_session`** will return the the passed `date` if `date` represents a session, or otherwise the nearest session in the passed `direction`.

In [4]:
# date is a session so `direction` is ignored
nys.date_to_session("2022-01-04", direction="next")

Timestamp('2022-01-04 00:00:00')

In [5]:
nys.date_to_session("2022-01-01", direction="next")

Timestamp('2022-01-03 00:00:00', freq='C')

In [6]:
nys.date_to_session("2022-01-01", direction="previous")

Timestamp('2021-12-31 00:00:00', freq='C')

A ValueError is raised if `direction` is not passed and `date` is not a session.

In [ ]:
nys.date_to_session("2022-01-01")

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [7], in <cell line: 1>()
----> 1 nys.date_to_session("2022-01-01")

ValueError: `date` '2022-01-01 00:00:00' does not represent a session. Consider passing a `direction`.
```

### Methods that query a Session

The argument of methods in this section takes a `Session` type.

The following methods can be used to return the **open, close** and **break times** of a session...

In [8]:
# for reference (all times are UTC)
session = "2022-01-04"
hkg.schedule.loc[[session]]

,open,break_start,break_end,close
2022-01-04,2022-01-04 01:30:00+00:00,2022-01-04 04:00:00+00:00,2022-01-04 05:00:00+00:00,2022-01-04 08:00:00+00:00


In [9]:
hkg.session_open(session), hkg.session_close(session)

(Timestamp('2022-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2022-01-04 08:00:00+0000', tz='UTC'))

In [10]:
# or
hkg.session_open_close(session)

(Timestamp('2022-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2022-01-04 08:00:00+0000', tz='UTC'))

In [11]:
hkg.session_break_start(session), hkg.session_break_end(session)

(Timestamp('2022-01-04 04:00:00+0000', tz='UTC'),
 Timestamp('2022-01-04 05:00:00+0000', tz='UTC'))

In [12]:
# or
hkg.session_break_start_end(session)

(Timestamp('2022-01-04 04:00:00+0000', tz='UTC'),
 Timestamp('2022-01-04 05:00:00+0000', tz='UTC'))

In [13]:
# in local time
hkg.session_open(session).tz_convert(hkg.tz)

Timestamp('2022-01-04 09:30:00+0800', tz='Asia/Hong_Kong')

The **`session_*_minute`** methods return a boundary trading minute for a session or subsession.

In [14]:
hkg.session_first_minute(session), hkg.session_last_minute(session)

(Timestamp('2022-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2022-01-04 07:59:00+0000', tz='UTC'))

In [15]:
hkg.session_last_am_minute(session), hkg.session_first_pm_minute(session)

(Timestamp('2022-01-04 03:59:00+0000', tz='UTC'),
 Timestamp('2022-01-04 05:00:00+0000', tz='UTC'))

In [16]:
# or get first and last minutes together...
hkg.session_first_last_minute(session)

(Timestamp('2022-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2022-01-04 07:59:00+0000', tz='UTC'))

In [17]:
# NB with "right" side...
hkg_right = xcals.get_calendar("XHKG", side="right")
hkg_right.session_first_last_minute(session)

(Timestamp('2022-01-04 01:31:00+0000', tz='UTC'),
 Timestamp('2022-01-04 08:00:00+0000', tz='UTC'))

**`session_has_break`** to query if a session has a break.

In [18]:
# Hong Kong has a break, at least on `session`...
hkg.session_has_break(session)

True

In [19]:
# but New York does not
nys.session_has_break(session)

False

In [20]:
# ...
nys.schedule.loc[[session]]

,open,break_start,break_end,close
2022-01-04,2022-01-04 14:30:00+00:00,NaT,NaT,2022-01-04 21:00:00+00:00


**`next_session`** and **`previous_session`** do what they say on the tin.

In [21]:
# just to recall...
session

'2022-01-04'

In [22]:
hkg.previous_session(session)

Timestamp('2022-01-03 00:00:00', freq='C')

In [23]:
hkg.next_session(session)

Timestamp('2022-01-05 00:00:00', freq='C')

**`session_offset`** takes a 'count' argument which provides for offsetting a session by a given number of sessions. If `count` is -1 or 1 then the method behaves as `previous_session` and `next_session` respectively.

In [24]:
nys.session_offset(session, count=-1), nys.session_offset(session, count=1)

(Timestamp('2022-01-03 00:00:00', freq='C'),
 Timestamp('2022-01-05 00:00:00', freq='C'))

In [25]:
nys.session_offset(session, -2), nys.session_offset(session, 2)

(Timestamp('2021-12-31 00:00:00', freq='C'),
 Timestamp('2022-01-06 00:00:00', freq='C'))

In [26]:
nys.session_offset(session, -50), nys.session_offset(session, 50)

(Timestamp('2021-10-22 00:00:00', freq='C'),
 Timestamp('2022-03-17 00:00:00', freq='C'))

In [27]:
# going nowhere...
nys.session_offset(session, 0)

Timestamp('2022-01-04 00:00:00', freq='C')

**`session_minutes`** returns an index of all the trading minutes of the passed session.

In [28]:
hkg.session_minutes(session)

DatetimeIndex(['2022-01-04 01:30:00+00:00', '2022-01-04 01:31:00+00:00',
               '2022-01-04 01:32:00+00:00', '2022-01-04 01:33:00+00:00',
               '2022-01-04 01:34:00+00:00', '2022-01-04 01:35:00+00:00',
               '2022-01-04 01:36:00+00:00', '2022-01-04 01:37:00+00:00',
               '2022-01-04 01:38:00+00:00', '2022-01-04 01:39:00+00:00',
               ...
               '2022-01-04 07:50:00+00:00', '2022-01-04 07:51:00+00:00',
               '2022-01-04 07:52:00+00:00', '2022-01-04 07:53:00+00:00',
               '2022-01-04 07:54:00+00:00', '2022-01-04 07:55:00+00:00',
               '2022-01-04 07:56:00+00:00', '2022-01-04 07:57:00+00:00',
               '2022-01-04 07:58:00+00:00', '2022-01-04 07:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=330, freq=None)

### Methods that query a Minute

The argument of methods in this section takes a `Minute` type.

**`is_trading_minute`** returns a boolean indicating if a minute is a trading minute.

In [29]:
# for reference
hkg.schedule.loc[["2022-01-04"]]

,open,break_start,break_end,close
2022-01-04,2022-01-04 01:30:00+00:00,2022-01-04 04:00:00+00:00,2022-01-04 05:00:00+00:00,2022-01-04 08:00:00+00:00


In [30]:
hkg.is_trading_minute("2022-01-04 01:25")

False

In [31]:
hkg.is_trading_minute("2022-01-04 01:35")

True

In [32]:
break_minute = "2022-01-04 04:35"
hkg.is_trading_minute(break_minute)

False

**`is_break_minute`** returns a boolean indicating if a minute lies within a lunch break. Break minutes lie between the last trading minute of the morning subsession and first trading minute of the afternoon subsession, exclusive of both (a break minute cannot be a trading minute).

In [33]:
hkg.is_break_minute(break_minute)

True

**`is_open_on_minute`** can take an optional `include_breaks` argument. If `include_breaks` is False (the default) then the method behaves in the same way as `is_trading_minute`. If `include_breaks` is True then the method will return True if `minute` is either a trading minute or a break minute (otherwise returns False).

In [34]:
hkg.is_open_on_minute(break_minute)

False

In [35]:
hkg.is_open_on_minute(break_minute, ignore_breaks=True)

True

**`minute_to_trading_minute`** will return the passed `minute` if `minute` is a trading minute, or otherwise the closest minute in the passed `direction`.

In [36]:
# define a trading minute
trading_minute = nys.first_minutes["2010-10-27"] + pd.Timedelta(45, "T")
trading_minute, nys.is_trading_minute(trading_minute)

(Timestamp('2010-10-27 14:15:00+0000', tz='UTC'), True)

In [37]:
# minute is a trading_minute, so `direction` ignored
nys.minute_to_trading_minute(trading_minute, direction="next")

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [38]:
# define a non-trading minute
non_trading_minute = nys.first_minutes["2010-10-27"] - pd.Timedelta(3, "T")
non_trading_minute, nys.is_trading_minute(non_trading_minute)

(Timestamp('2010-10-27 13:27:00+0000', tz='UTC'), False)

In [39]:
nys.minute_to_trading_minute(non_trading_minute, direction="next")

Timestamp('2010-10-27 13:30:00+0000', tz='UTC')

In [40]:
nys.minute_to_trading_minute(non_trading_minute, direction="previous")

Timestamp('2010-10-26 19:59:00+0000', tz='UTC')

A ValueError is raised if `direction` is not passed and `minute` is not a trading minute.

In [ ]:
nys.minute_to_trading_minute(non_trading_minute)

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [41], in <cell line: 1>()
----> 1 nys.minute_to_trading_minute(non_trading_minute)

ValueError: `minute` '2010-10-27 13:27:00+00:00' is not a trading minute. Consider passing `direction` as 'next' or 'previous'.
```

**`previous_open`**, **`next_open`**, **`previous_close`** and **`next_close`** will return the previous or next open or close relative to the passed `minute`. NB Which minutes are treated as trading minutes is irrelevant.

In [42]:
# for reference
nys.schedule.loc["2022-01-04":"2022-01-05"]

,open,break_start,break_end,close
2022-01-04,2022-01-04 14:30:00+00:00,NaT,NaT,2022-01-04 21:00:00+00:00
2022-01-05,2022-01-05 14:30:00+00:00,NaT,NaT,2022-01-05 21:00:00+00:00


In [43]:
open_05 = nys.session_open("2022-01-05")
open_05

Timestamp('2022-01-05 14:30:00+0000', tz='UTC')

In [44]:
nys.previous_open(open_05), nys.next_open(open_05)

(Timestamp('2022-01-04 14:30:00+0000', tz='UTC'),
 Timestamp('2022-01-06 14:30:00+0000', tz='UTC'))

In [45]:
nys.previous_open(open_05 + one_minute)

Timestamp('2022-01-05 14:30:00+0000', tz='UTC')

In [46]:
nys.previous_close(open_05), nys.next_close(open_05)

(Timestamp('2022-01-04 21:00:00+0000', tz='UTC'),
 Timestamp('2022-01-05 21:00:00+0000', tz='UTC'))

**`previous_minute`** and **`next_minute`** return the first trading minute prior to / following `minute`.

In [47]:
minute = nys.first_minutes["2022-01-05"]
minute

Timestamp('2022-01-05 14:30:00+0000', tz='UTC')

In [48]:
nys.previous_minute(minute), nys.next_minute(minute)

(Timestamp('2022-01-04 20:59:00+0000', tz='UTC'),
 Timestamp('2022-01-05 14:31:00+0000', tz='UTC'))

In [49]:
minute -= (one_minute * 2)
minute

Timestamp('2022-01-05 14:28:00+0000', tz='UTC')

In [50]:
nys.previous_minute(minute), nys.next_minute(minute)

(Timestamp('2022-01-04 20:59:00+0000', tz='UTC'),
 Timestamp('2022-01-05 14:30:00+0000', tz='UTC'))

**`minute_offset`** takes a 'count' argument which provides for offsetting a trading minute by the given number of minutes. If count is -1 or 1 then the method behaves as `previous_minute` and `next_minute` respectively.

In [51]:
# recalling
trading_minute

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [52]:
nys.minute_offset(trading_minute, count=-1), nys.minute_offset(trading_minute, 1)

(Timestamp('2010-10-27 14:14:00+0000', tz='UTC'),
 Timestamp('2010-10-27 14:16:00+0000', tz='UTC'))

In [53]:
nys.minute_offset(trading_minute, -5), nys.minute_offset(trading_minute, 5)

(Timestamp('2010-10-27 14:10:00+0000', tz='UTC'),
 Timestamp('2010-10-27 14:20:00+0000', tz='UTC'))

In [54]:
nys.minute_offset(trading_minute, -45), nys.minute_offset(trading_minute, 344)

(Timestamp('2010-10-27 13:30:00+0000', tz='UTC'),
 Timestamp('2010-10-27 19:59:00+0000', tz='UTC'))

In [55]:
nys.minute_offset(trading_minute, -46), nys.minute_offset(trading_minute, 345)

(Timestamp('2010-10-26 19:59:00+0000', tz='UTC'),
 Timestamp('2010-10-28 13:30:00+0000', tz='UTC'))

In [56]:
nys.minute_offset(trading_minute, -3000), nys.minute_offset(trading_minute, 3000)

(Timestamp('2010-10-15 16:15:00+0000', tz='UTC'),
 Timestamp('2010-11-05 18:45:00+0000', tz='UTC'))

In [57]:
# and of course
nys.minute_offset(trading_minute, 0)

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

**`minute_to_session`** returns the session associated with `minute`. If `minute` is a trading minute or a break minute then this will be the session of which the minute is a trading/break minute.

In [58]:
nys.minute_to_session(trading_minute)

Timestamp('2010-10-27 00:00:00', freq='C')

If `minute` is not a trading or break minute then the return is determined by the optional `direction` parameter. If `direction` is "next" the return will be the closest session after `minute`, whilst if `direction` is "previous" the return will be the closest session before `minute`.

In [59]:
# for reference
non_trading_minute

Timestamp('2010-10-27 13:27:00+0000', tz='UTC')

In [60]:
# default `direction` is "next"
nys.minute_to_session(non_trading_minute)

Timestamp('2010-10-27 00:00:00', freq='C')

In [61]:
nys.minute_to_session(non_trading_minute, direction="previous")

Timestamp('2010-10-26 00:00:00', freq='C')

`direction` can also take "none", in which case `minute` is required to be a trading or break minute. To the contrary a ValueError is raised:

In [ ]:
nys.minute_to_session(non_trading_minute, direction="none")

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [62], in <cell line: 1>()
----> 1 nys.minute_to_session(non_trading_minute, direction="none")

ValueError: `minute` '2010-10-27 13:27:00+00:00' is not a trading minute. Consider passing `direction` as 'next' or 'previous'.
```

**`minute_to_past_session`** returns a session that closed before the given `minute`. If the passed `minute` is a trading minute then the method will NOT return the session of which `minute` is a trading minute, but rather a prior session.

In [63]:
# recalling
trading_minute

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [64]:
nys.minute_to_past_session(trading_minute)

Timestamp('2010-10-26 00:00:00', freq='C')

Note how the above differs to `minute_to_session`...

In [65]:
nys.minute_to_session(trading_minute)

Timestamp('2010-10-27 00:00:00', freq='C')

`minute_to_past_session` can take an optional `count` argument (default 1) to offset the past session returned.

In [66]:
# as default
nys.minute_to_past_session(trading_minute, count=1)

Timestamp('2010-10-26 00:00:00', freq='C')

In [67]:
nys.minute_to_past_session(trading_minute, count=2)

Timestamp('2010-10-25 00:00:00', freq='C')

In [68]:
nys.minute_to_past_session(trading_minute, count=3)
# jumping a weekend...

Timestamp('2010-10-22 00:00:00', freq='C')

**`minute_to_future_session`** works in the same way as **`minute_to_past_session`** albeit looking fowards.

In [69]:
nys.minute_to_future_session(trading_minute)
# default count is 1

Timestamp('2010-10-28 00:00:00', freq='C')

In [70]:
nys.minute_to_future_session(trading_minute, count=2)

Timestamp('2010-10-29 00:00:00', freq='C')

**`minute_offset_by_sessions`** offsets a trading minute by a given number of sessions, by default 1.

In [71]:
# recalling again
trading_minute

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [72]:
nys.minute_offset_by_sessions(trading_minute)
# default count is 1

Timestamp('2010-10-28 14:15:00+0000', tz='UTC', freq='C')

In [73]:
nys.minute_offset_by_sessions(trading_minute, count=-2)
# offseting 'backwards'

Timestamp('2010-10-25 14:15:00+0000', tz='UTC', freq='C')

If a would-be offset minute is not a minute of the offset session then...

If the would-be offset minute is later than the offset session's close, the offset session's last trading minute is returned.

In [74]:
# for reference
nys.schedule.loc["2020-12-23":"2020-12-28", "close"].to_frame()

,close
2020-12-23,2020-12-23 21:00:00+00:00
2020-12-24,2020-12-24 18:00:00+00:00
2020-12-28,2020-12-28 21:00:00+00:00


In [75]:
nys.minute_offset_by_sessions("2020-12-23 20:22", 1)

Timestamp('2020-12-24 17:59:00+0000', tz='UTC')

If the would-be offset minute is earlier than the offset session's open, the offset session's first trading minute is returned.

In [76]:
# for reference
hkg.schedule.loc["'2011-03-03":"2011-03-08", "open"].to_frame()

,open
2011-03-03,2011-03-03 02:00:00+00:00
2011-03-04,2011-03-04 02:00:00+00:00
2011-03-07,2011-03-07 01:30:00+00:00
2011-03-08,2011-03-08 01:30:00+00:00


In [77]:
hkg.minute_offset_by_sessions("2011-03-08 01:47", count=-2)

Timestamp('2011-03-04 02:00:00+0000', tz='UTC')

If the would-be offset minute is a break minute, the last pre-break trading minute is returned.

In [78]:
# for reference
hkg.schedule.loc["2010-12-23":"2010-12-24"]

,open,break_start,break_end,close
2010-12-23,2010-12-23 02:00:00+00:00,2010-12-23 04:00:00+00:00,2010-12-23 05:00:00+00:00,2010-12-23 08:00:00+00:00
2010-12-24,2010-12-24 02:00:00+00:00,NaT,NaT,2010-12-24 04:30:00+00:00


In [79]:
hkg.minute_offset_by_sessions("2010-12-24 04:15", count=-1)

Timestamp('2010-12-23 03:59:00+0000', tz='UTC')

### Methods that evaluate an index of contiguous trading minutes

**`minutes_in_range`** returns all trading minutes within and inclusive of `start` and `end`. The parameters take `Minute` type (i.e. they do not need to represent actual trading minutes).

In [80]:
# recalling...
hkg.schedule.loc[["2022-01-04"]]

,open,break_start,break_end,close
2022-01-04,2022-01-04 01:30:00+00:00,2022-01-04 04:00:00+00:00,2022-01-04 05:00:00+00:00,2022-01-04 08:00:00+00:00


In [81]:
hkg.minutes_in_range("2022-01-04 03:55", "2022-01-04 05:05")

DatetimeIndex(['2022-01-04 03:55:00+00:00', '2022-01-04 03:56:00+00:00',
               '2022-01-04 03:57:00+00:00', '2022-01-04 03:58:00+00:00',
               '2022-01-04 03:59:00+00:00', '2022-01-04 05:00:00+00:00',
               '2022-01-04 05:01:00+00:00', '2022-01-04 05:02:00+00:00',
               '2022-01-04 05:03:00+00:00', '2022-01-04 05:04:00+00:00',
               '2022-01-04 05:05:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [82]:
# `start` does not have to represent a trading minute
hkg.minutes_in_range("2022-01-04 04:30", "2022-01-04 05:05")

DatetimeIndex(['2022-01-04 05:00:00+00:00', '2022-01-04 05:01:00+00:00',
               '2022-01-04 05:02:00+00:00', '2022-01-04 05:03:00+00:00',
               '2022-01-04 05:04:00+00:00', '2022-01-04 05:05:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Alternatively, **`minutes_window`** can be used to create an index of trading minutes of a defined length, anchored by the passed `minute`.

In [83]:
hkg.minutes_window("2022-01-04 07:55", count=10)

DatetimeIndex(['2022-01-04 07:55:00+00:00', '2022-01-04 07:56:00+00:00',
               '2022-01-04 07:57:00+00:00', '2022-01-04 07:58:00+00:00',
               '2022-01-04 07:59:00+00:00', '2022-01-05 01:30:00+00:00',
               '2022-01-05 01:31:00+00:00', '2022-01-05 01:32:00+00:00',
               '2022-01-05 01:33:00+00:00', '2022-01-05 01:34:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Or to work backwards from `minute`...

In [84]:
hkg.minutes_window("2022-01-04 07:55", count=-10)

DatetimeIndex(['2022-01-04 07:46:00+00:00', '2022-01-04 07:47:00+00:00',
               '2022-01-04 07:48:00+00:00', '2022-01-04 07:49:00+00:00',
               '2022-01-04 07:50:00+00:00', '2022-01-04 07:51:00+00:00',
               '2022-01-04 07:52:00+00:00', '2022-01-04 07:53:00+00:00',
               '2022-01-04 07:54:00+00:00', '2022-01-04 07:55:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### Methods that query multiple TradingMinute

**`minutes_to_sessions`** does for multiple TradingMinute what `minute_to_session` does for one. However, this method is limited by requiring every minute of the passed `minutes` parameter to be a trading minute (i.e. there's no `direction` parameter available for this one). Also, whilst minutes do not need to be contiguous, they do need to be in ascending order.

In [85]:
# set up a minute index to pass as `minutes`..
ser = nys.first_minutes[-200:-195] + (one_minute * 5)
ser

2022-09-02   2022-09-02 13:35:00+00:00
2022-09-06   2022-09-06 13:35:00+00:00
2022-09-07   2022-09-07 13:35:00+00:00
2022-09-08   2022-09-08 13:35:00+00:00
2022-09-09   2022-09-09 13:35:00+00:00
Freq: C, Name: first_minutes, dtype: datetime64[ns, UTC]

In [86]:
# still setting up 'minutes'
mins = pd.DatetimeIndex(ser)
mins = mins[:1].union(mins[-2:], sort=False)
mins

DatetimeIndex(['2022-09-02 13:35:00+00:00', '2022-09-08 13:35:00+00:00',
               '2022-09-09 13:35:00+00:00'],
              dtype='datetime64[ns, UTC]', name='first_minutes', freq=None)

In [87]:
nys.minutes_to_sessions(mins)

DatetimeIndex(['2022-09-02', '2022-09-08', '2022-09-09'], dtype='datetime64[ns]', freq=None)

In [88]:
# NB `minutes` can also be passed as a Series...
nys.minutes_to_sessions(ser)

DatetimeIndex(['2022-09-02', '2022-09-06', '2022-09-07', '2022-09-08',
               '2022-09-09'],
              dtype='datetime64[ns]', freq=None)

### Methods that evaluate an index of contiguous sessions

**`sessions_in_range`** does for sessions what `minutes_in_range` does for minutes. Returns an index of all the sessions between `start` and `end`. Parameters take `Date` type (i.e. they do not need to represent actual sessions).

In [89]:
nys.sessions_in_range("2022", "2022-12-31")

DatetimeIndex(['2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06',
               '2022-01-07', '2022-01-10', '2022-01-11', '2022-01-12',
               '2022-01-13', '2022-01-14',
               ...
               '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21',
               '2022-12-22', '2022-12-23', '2022-12-27', '2022-12-28',
               '2022-12-29', '2022-12-30'],
              dtype='datetime64[ns]', length=251, freq='C')

**`sessions_window`** holds no surprises if you're familiar with `minutes_window`...

In [90]:
nys.sessions_window("2021-12-23", 3)

DatetimeIndex(['2021-12-23', '2021-12-27', '2021-12-28'], dtype='datetime64[ns]', freq='C')

In [91]:
nys.sessions_window("2021-12-23", -3)

DatetimeIndex(['2021-12-21', '2021-12-22', '2021-12-23'], dtype='datetime64[ns]', freq='C')

### Methods that query a range of dates

The methods in this section query sessions that fall within the range of dates from `start` through `end` (inclusive of both). Both parameters take a `Date` (i.e the passed values can but do not have to represent an actual session).

In [92]:
start, end = "2021-12-23", "2021-12-29"
# for reference
nys.schedule.loc[start:end]

,open,break_start,break_end,close
2021-12-23,2021-12-23 14:30:00+00:00,NaT,NaT,2021-12-23 21:00:00+00:00
2021-12-27,2021-12-27 14:30:00+00:00,NaT,NaT,2021-12-27 21:00:00+00:00
2021-12-28,2021-12-28 14:30:00+00:00,NaT,NaT,2021-12-28 21:00:00+00:00
2021-12-29,2021-12-29 14:30:00+00:00,NaT,NaT,2021-12-29 21:00:00+00:00


**`sessions_distance`** returns the number of sessions in the range.

In [93]:
nys.sessions_distance(start, end)

4

**`sessions_minutes`** returns an index comprised of the trading minutes of all sessions in the given range.

In [94]:
nys.sessions_minutes(start, end)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:31:00+00:00',
               '2021-12-23 14:32:00+00:00', '2021-12-23 14:33:00+00:00',
               '2021-12-23 14:34:00+00:00', '2021-12-23 14:35:00+00:00',
               '2021-12-23 14:36:00+00:00', '2021-12-23 14:37:00+00:00',
               '2021-12-23 14:38:00+00:00', '2021-12-23 14:39:00+00:00',
               ...
               '2021-12-29 20:50:00+00:00', '2021-12-29 20:51:00+00:00',
               '2021-12-29 20:52:00+00:00', '2021-12-29 20:53:00+00:00',
               '2021-12-29 20:54:00+00:00', '2021-12-29 20:55:00+00:00',
               '2021-12-29 20:56:00+00:00', '2021-12-29 20:57:00+00:00',
               '2021-12-29 20:58:00+00:00', '2021-12-29 20:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=1560, freq=None)

**`sessions_minutes_count`** returns just the number of trading minutes corresponding to the sessions.

In [95]:
nys.sessions_minutes_count(start, end)

1560

In [96]:
# i.e. `sessions_minutes_count` behaves as...
len(nys.sessions_minutes(start, end))

1560

`sessions_opens` and `sessions_closes` were **deprecated** in 4.0. Instead query the analogous properties:

In [97]:
nys.opens[start:end]

2021-12-23   2021-12-23 14:30:00+00:00
2021-12-27   2021-12-27 14:30:00+00:00
2021-12-28   2021-12-28 14:30:00+00:00
2021-12-29   2021-12-29 14:30:00+00:00
Freq: C, Name: open, dtype: datetime64[ns, UTC]

In [98]:
nys.closes[start:end]

2021-12-23   2021-12-23 21:00:00+00:00
2021-12-27   2021-12-27 21:00:00+00:00
2021-12-28   2021-12-28 21:00:00+00:00
2021-12-29   2021-12-29 21:00:00+00:00
Freq: C, Name: close, dtype: datetime64[ns, UTC]

### Methods that query a time

`is_open_at_time` is rather unique in that it is not concerned with any specific `Minute` or `Date` but rather only if the exchange is considered open as at a given instance. The calendar's side has no effect on the return. Consequently, 
even if the calendar's side is "both" it's possible to query if the market is open as at a time specified with second or greater accuracy.

In [99]:
hkg_both = xcals.get_calendar("XHKG", side="both")

In [100]:
# recalling...
hkg.schedule.loc[["2022-01-04"]]

,open,break_start,break_end,close
2022-01-04,2022-01-04 01:30:00+00:00,2022-01-04 04:00:00+00:00,2022-01-04 05:00:00+00:00,2022-01-04 08:00:00+00:00


In [101]:
timestamp = pd.Timestamp("2022-01-04 07:59:59")
hkg_both.is_open_at_time(timestamp)

True

Note that unlike other methods, the input has to be an instance of `pd.Timestamp`. If passed as timezone-naive (as here) it will be assumed to represent UTC.

`is_open_at_time` can take a 'side' option to determine if the exchange will be considered open or closed on a session's bounds:
* **"left"** (default) - treat exchange as open on session open and any break-end, treat as closed on session close and any break-start.
* **"right"** - treat exchange as open on session close and any break-start, treat as closed on session open and any break-end.
* **"both"** - treat exchange as open on all of session open, close and any break-start and break-end.
* **"neither"** - treat exchange as closed on all of session open, close and any break-start and break-end.

It can also take an `ignore_breaks` options which, as for `is_open_on_minute`, will treat the exchange as open during any break.

In [102]:
sides = ("left", "right", "both", "neither")
timestamp = pd.Timestamp("2022-01-04 01:30:00")
[ hkg.is_open_at_time(timestamp, side=side) for side in sides ]

[True, False, True, False]

In [103]:
timestamp = pd.Timestamp("2022-01-04 04:00:00")
[ hkg.is_open_at_time(timestamp, side=side) for side in sides ]

[False, True, True, False]

In [104]:
[ hkg.is_open_at_time(timestamp, side, ignore_breaks=True) for side in sides ]

[True, True, True, True]

In [105]:
timestamp = pd.Timestamp("2022-01-04 03:59:59")
[ hkg.is_open_at_time(timestamp, side=side) for side in sides ]

[True, True, True, True]

In [106]:
timestamp = pd.Timestamp("2022-01-04 04:00:01")
[ hkg.is_open_at_time(timestamp, side=side) for side in sides ]

[False, False, False, False]

In [107]:
[ hkg.is_open_at_time(timestamp, side, ignore_breaks=False) for side in sides ]

[False, False, False, False]

### **`trading_index`**  
Saving the best for last, `trading_index` provides for creating a trading index of given `period` over a range of sessions. It's options provide so much flexibility that it's got it's own tutorial at [trading_index.ipynb](./trading_index.ipynb). 

Here's a taster...

In [108]:
# recalling
start, end

('2021-12-23', '2021-12-29')

In [109]:
nys.trading_index(start, end, "15min", intervals=False)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:45:00+00:00',
               '2021-12-23 15:00:00+00:00', '2021-12-23 15:15:00+00:00',
               '2021-12-23 15:30:00+00:00', '2021-12-23 15:45:00+00:00',
               '2021-12-23 16:00:00+00:00', '2021-12-23 16:15:00+00:00',
               '2021-12-23 16:30:00+00:00', '2021-12-23 16:45:00+00:00',
               ...
               '2021-12-29 18:30:00+00:00', '2021-12-29 18:45:00+00:00',
               '2021-12-29 19:00:00+00:00', '2021-12-29 19:15:00+00:00',
               '2021-12-29 19:30:00+00:00', '2021-12-29 19:45:00+00:00',
               '2021-12-29 20:00:00+00:00', '2021-12-29 20:15:00+00:00',
               '2021-12-29 20:30:00+00:00', '2021-12-29 20:45:00+00:00'],
              dtype='datetime64[ns, UTC]', length=104, freq=None)

In [110]:
nys.trading_index(start, start, "24min", closed="right", intervals=False)

DatetimeIndex(['2021-12-23 14:54:00+00:00', '2021-12-23 15:18:00+00:00',
               '2021-12-23 15:42:00+00:00', '2021-12-23 16:06:00+00:00',
               '2021-12-23 16:30:00+00:00', '2021-12-23 16:54:00+00:00',
               '2021-12-23 17:18:00+00:00', '2021-12-23 17:42:00+00:00',
               '2021-12-23 18:06:00+00:00', '2021-12-23 18:30:00+00:00',
               '2021-12-23 18:54:00+00:00', '2021-12-23 19:18:00+00:00',
               '2021-12-23 19:42:00+00:00', '2021-12-23 20:06:00+00:00',
               '2021-12-23 20:30:00+00:00', '2021-12-23 20:54:00+00:00',
               '2021-12-23 21:18:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [111]:
nys.trading_index(start, start, "24min", force_close=True, intervals=False)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:54:00+00:00',
               '2021-12-23 15:18:00+00:00', '2021-12-23 15:42:00+00:00',
               '2021-12-23 16:06:00+00:00', '2021-12-23 16:30:00+00:00',
               '2021-12-23 16:54:00+00:00', '2021-12-23 17:18:00+00:00',
               '2021-12-23 17:42:00+00:00', '2021-12-23 18:06:00+00:00',
               '2021-12-23 18:30:00+00:00', '2021-12-23 18:54:00+00:00',
               '2021-12-23 19:18:00+00:00', '2021-12-23 19:42:00+00:00',
               '2021-12-23 20:06:00+00:00', '2021-12-23 20:30:00+00:00',
               '2021-12-23 20:54:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [112]:
index = nys.trading_index(start, end, "20min", intervals=True)
index

IntervalIndex([[2021-12-23 14:30:00, 2021-12-23 14:50:00), [2021-12-23 14:50:00, 2021-12-23 15:10:00), [2021-12-23 15:10:00, 2021-12-23 15:30:00), [2021-12-23 15:30:00, 2021-12-23 15:50:00), [2021-12-23 15:50:00, 2021-12-23 16:10:00) ... [2021-12-29 19:30:00, 2021-12-29 19:50:00), [2021-12-29 19:50:00, 2021-12-29 20:10:00), [2021-12-29 20:10:00, 2021-12-29 20:30:00), [2021-12-29 20:30:00, 2021-12-29 20:50:00), [2021-12-29 20:50:00, 2021-12-29 21:10:00)], dtype='interval[datetime64[ns, UTC], left]')

In [113]:
# just to see it clearer...
pd.DataFrame(dict(left_side=index.left, right_side=index.right), index=index)

,left_side,right_side
"[2021-12-23 14:30:00, 2021-12-23 14:50:00)",2021-12-23 14:30:00+00:00,2021-12-23 14:50:00+00:00
"[2021-12-23 14:50:00, 2021-12-23 15:10:00)",2021-12-23 14:50:00+00:00,2021-12-23 15:10:00+00:00
"[2021-12-23 15:10:00, 2021-12-23 15:30:00)",2021-12-23 15:10:00+00:00,2021-12-23 15:30:00+00:00
"[2021-12-23 15:30:00, 2021-12-23 15:50:00)",2021-12-23 15:30:00+00:00,2021-12-23 15:50:00+00:00
"[2021-12-23 15:50:00, 2021-12-23 16:10:00)",2021-12-23 15:50:00+00:00,2021-12-23 16:10:00+00:00
...,...,...
"[2021-12-29 19:30:00, 2021-12-29 19:50:00)",2021-12-29 19:30:00+00:00,2021-12-29 19:50:00+00:00
"[2021-12-29 19:50:00, 2021-12-29 20:10:00)",2021-12-29 19:50:00+00:00,2021-12-29 20:10:00+00:00
"[2021-12-29 20:10:00, 2021-12-29 20:30:00)",2021-12-29 20:10:00+00:00,2021-12-29 20:30:00+00:00
"[2021-12-29 20:30:00, 2021-12-29 20:50:00)",2021-12-29 20:30:00+00:00,2021-12-29 20:50:00+00:00
